In [10]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd
from pyspark.sql.functions import *

In [9]:
pip install pymysql

   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   ------------------ --------------------- 20.5/44.8 kB 330.3 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/44.8 kB 326.8 kB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 kB 276.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
product_path='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/Zubaidnizami-fp-ai-081beb29d155/Code/Database/product.csv'
cosin_accessories='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/cosin_similarity_registry/tag_collection.csv'
cosin_clothing='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/cosin_similarity_registry/collection_tag.csv'#'C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/cosin_simmilarity1.csv'
cosin_result_path3='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/cosin_similarity_registry/collection_tag_FE.csv'

collection_category='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/Database/collection_category.csv'
collection_category_tag='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/Database/collection_category_tag.csv'
product_tag='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/Database/product_tag.csv'
product='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/Database/product.csv'
tag_cloud='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/Database/tag_cloud.csv'
feature_accesories='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/feature_registry/featurelist_22_1_2024_accessories.csv'
feature_clothing='C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/feature_registry/featurelist_12_1_2024_clothing.csv'

In [22]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .config("spark.driver.memory", "15g") \
    .getOrCreate()



In [20]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)
cosin_accessories_df=spark.read.format('csv').option('header',True) \
.load(cosin_accessories)
cosin_clothing_df=spark.read.format('csv').option('header',True) \
.load(cosin_clothing)
cosin_result_path_df3=spark.read.format('csv').option('header',True) \
.load(cosin_result_path3)
collection_category_df=spark.read.format('csv').option('header',True) \
.load(collection_category)
collection_category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag)
tag_cloud_df=spark.read.format('csv').option('header',True) \
.load(tag_cloud)
product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag)


feature_accesories_df=spark.read.format('csv').option('header',True) \
.load(feature_accesories)
feature_clothing_df=spark.read.format('csv').option('header',True) \
.load(feature_clothing)








AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/D:/PROJECTS/Recomendation_System_FP/Code/Database/product.csv.

In [13]:
import numpy as np
def feature_list(df=None):
    """
    Extract feature lists from the input DataFrame.

    Parameters:
    - df (pyspark.sql.DataFrame): Input DataFrame containing feature columns.

    Returns:
    dict: A dictionary containing different feature lists.

    Feature Lists:
    - 'category_collection_FE': List of values extracted from the 'category_collection_FE' column.
    - 'categoey_collection_tag_L': List of values extracted from the 'category_collection_tag_L' column.
    - 'tag_cloud': List of values extracted from the 'tag_cloud' column.
    """
    # Extract values from 'category_collection_FE', 'category_collection_tag_L', and 'tag_cloud' columns
    category_collection_FE = [i[0] for i in df.select('category_collection_FE').collect()]
    category_collection_tag_L = [i[0] for i in df.select('category_collection_tag_L').collect()]
    tag_cloud = [i[0] for i in df.select('tag_cloud').collect()]

    # Return a dictionary containing the extracted feature lists
    return {
        'category_collection_FE': category_collection_FE,
        'categoey_collection_tag_L': category_collection_tag_L,
        'tag_cloud': tag_cloud
    }

In [16]:
feature_accesories_df=feature_list(feature_accesories_df)
feature_clothing_df=feature_list(feature_clothing_df)

In [9]:
def duplicate_col(data=None):
  """
    Duplicate columns in a DataFrame by appending '_duplicate_' and the index to the column name.

    Parameters:
    - data (pd.DataFrame): The input DataFrame with columns to be duplicated.

    Returns:
    pd.DataFrame: A new DataFrame with duplicated columns.
    """
  df_cols = data.columns
  duplicate_col_index = [idx for idx,
  val in enumerate(df_cols) if val in df_cols[:idx]]
  for i in duplicate_col_index:
    df_cols[i] = df_cols[i] + '_duplicate_'+ str(i)
  data = data.toDF(*df_cols)
  return data

In [41]:


def combine_cosin_df(cosin_accessories_df,cosin_clothing_df,product_tag_df,tag_cloud_df,Product_df,collection_category_tag_df,feature_accesories_df,feature_clothing_df):


    product_tag_name_df=product_tag_df.join(tag_cloud_df,product_tag_df.tag_id==tag_cloud_df.id)
    product_tag_name_df=product_tag_name_df.join(Product_df,Product_df.product_id==product_tag_name_df.product_id,'inner')
    product_tag_name_df=duplicate_col(product_tag_name_df)
    product_tag_name_df=product_tag_name_df.select(['product_id','tag_id','id','tag_name','product_title'])
    product_tag_name_collection_df=product_tag_name_df.join(collection_category_tag_df,product_tag_name_df.tag_id==collection_category_tag_df.tag_id)
    product_accesories_ids=[  int(i) for i in np.unique(product_tag_name_collection_df.filter
    (product_tag_name_collection_df['category_id'].isin(
        
        feature_accesories_df['category_collection_FE']
        
    )).select('product_id').collect())]
    product_clothing_ids=[ int(i)  for i in np.unique(product_tag_name_collection_df.filter
    (product_tag_name_collection_df['category_id'].isin(
     feature_clothing_df['category_collection_FE']

    )).select('product_id').collect())]
    
    #--testing cosin csv--#
        # import pandas as pd
        # cosin_clothing_df.to_csv('cosin_clothing_df.csv')
        # cosin_accessories_df.to_csv('cosin_accessories_df.csv')
    #--testing cosin csv--#
    cosin_clothing_df=cosin_clothing_df[~cosin_clothing_df['Product_A'].isin(product_accesories_ids)]
    cosin_accessories_df=cosin_accessories_df[~cosin_accessories_df['Product_A'].isin(product_clothing_ids)]
    #--wrote for pyspark--#
    # cosin_accessories_df=cosin_accessories_df.filter(~cosin_accessories_df['Product_B'].isin(product_clothing_ids))  
    # cosin_clothing_df=cosin_clothing_df.filter(~cosin_clothing_df['Product_B'].isin(product_accesories_ids))
    # cosin_clothing_df=cosin_clothing_df.toPandas()
    # cosin_accessories_df=cosin_accessories_df.toPandas()
    #--wrote for pyspark--#
    print(cosin_accessories_df.head())
    print(cosin_clothing_df.head())
    merge_csv=cosin_accessories_df._append(cosin_clothing_df,ignore_index = True)  
    merge_csv=merge_csv[['cosin_score',  'Product_A'  ,'Product_B']]
    return merge_csv  

In [42]:
accesories_df=pd.read_csv('C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/cosin_accessories_df.csv')
clothing_df=pd.read_csv('C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/cosin_clothing_df.csv')
combine_cosin_df(accesories_df,clothing_df,product_tag_df,tag_cloud_df,Product_df,collection_category_tag_df,feature_accesories_df,feature_clothing_df)

   Unnamed: 0.1  Unnamed: 0  cosin_score  Product_A  Product_B
0             0           0     0.731022        829       1436
1             1           1     0.601156        829       6731
2             2           2     0.861249        829       2294
3             3           3     0.852268        829       1159
4             4           4     0.500400        829        296
   Unnamed: 0.1  Unnamed: 0  cosin_score  Product_A  Product_B
0             0           0     0.429471       1159       4032
1             1           1     0.526363       1159       3606
2             2           2     0.404225       1159        675
3             3           3     0.386409       1159       2904
4             4           4     1.000000       1159       4937
   Unnamed: 0.1  Unnamed: 0  cosin_score  Product_A  Product_B
0             0           0     0.731022        829       1436
1             1           1     0.601156        829       6731
2             2           2     0.861249        829    

In [10]:
# product_A=cosin_clothing_df.select('Product_A').collect()
# product_B=cosin_accessories_df.select('Product_A').collect()

In [11]:
product_A=np.unique(product_A)
product_B=np.unique(product_B)

In [12]:
print(len(product_A),
len(product_B))

4278 4278


In [ ]:
combine_csv=cosin_accessories_df.union(cosin_clothing_df)
combine_csv=combine_csv.dropDuplicates()

In [ ]:
combine_csv.show()

In [ ]:
combine_csv.write.options(header='True', delimiter=',') \
 .csv("C:/Users/Bionic Computer/Downloads/ubaidnizami-fp-ai/ubaidnizami-fp-ai-081beb29d155/Code/notebook")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/D:/PROJECTS/Recomendation_System_FP/Code/notebook already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
cosin_result_path_df1=combine_csv.toPandas()

Py4JJavaError: An error occurred while calling o5417.collectToPython.
: java.lang.OutOfMemoryError: Java heap space


In [ ]:
cosin_result_path_df1=combine_csv

In [ ]:
product_title='X FASHIONPASS MARCIE EARRINGS'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df1[cosin_result_path_df1['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

: 

In [ ]:
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df2[cosin_result_path_df2['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

3939,4407,7137,8167,2668,6832,8202,4815,8245,7534,8570,2006,5441,4286,7811,5091,2459,7998,5194,5943,8477,2591,5602,5032,6224,6845,8552,7641,3962,8081,1976,4627,2687,5860,3963,2844,8331,3323,7141,3623,8476,4811,4900,7733,8753,4946,7960,4285,8097,7804,7726,5172,7844,7803,1951,4331,4311,7725,3769,4810,1231,2328,7308,5940,8143,5382,8250,7661,6798,5410,5938,6283,8080,3961,6223,6678,2177,7826,4962,4108,4406,2254,8051,8637,6890,3192,7148,3217,5423,7602,8752,5381,8550,8750,3564,8664,4927,7997,5438,4143,8254,7535,4625,5736,180,4102,1531,316,6432,1450,5201,5719,5741,6975,6231,7619,5939,8316,8076,2549,8075,3759,5424,5422,5368,3878,5405,5288,5611,859,6225,6073,3440,1952,3423,5404,5369,5182,4963,7140,7986,2332,4978,7876,4210,7800,6540,4287,3205,3575,7802,7796,1949,5279,1614,7938,5089,8754,3385,557,8475,2001,915,6421,5858,6610,3749,3725,5281,8487,5126,5171,4145,2399,7172,7660,2707,5185,7943,8751,2114,2029,5131,6265,278,6115,3396,8054,8867,8400,4409,3422,8068,7124,8523,6397,6880,8660,1251


In [ ]:
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df3[cosin_result_path_df3['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

5444,2324,2940,7779,1668,7630,1200,2747,604,104,3599,978,3713,5342,7675,5973,3921,1557,7555,7874,8112,4763,1790,645,5506,1237,1301,1105,1078,8033,3473,7253,3538,1348,1201,8405,1028,7599,1714,1068,5446,7748,1206,1523,7276,5749,8403,1558,7870,8155,4188,7557,7608,281,461,741,1080,350,114,2897,540,759,8192,4958,1597,8032,7965,1768,3598,8102,490,24,7558,757,3356,8141,7744,5391,1778,289,8156,5448,7443,5447,8132,7254,1848,2924,8231,776,3596,3675,5670,8404,7783,3514,49,1661,4425,7610,7265,1242,2320,4181,4254,3710,7871,1233,1029,4251,1553,7206,4486,1350,1273,66,8191,8232,760,2730,8230,8401,5389,8506,7216,1699,6718,1781,2704,3594,5390,7264,8111,2746,3601,2732,5953,4316,145,2076,8188,6137,881,1779,2317,119,8128,8190,1329,8189


In [ ]:
import pandas as pd
cs1=pd.read_csv(cosin_result_path3)#configur.get('cosin_similarity_base_path','cs1'))#load_csv(spark_session,file_path=configur.get('cosin_similarity_base_path','cs1'))
cs2=pd.read_csv(cosin_result_path2)#configur.get('cosin_similarity_base_path','cs2'))#load_csv(spark=spark_session,file_path=configur.get('cosin_similarity_base_path','cs2'))
cs3=pd.read_csv(cosin_result_path1)

In [ ]:
product_title='X CAMELIA FARHOODI THE LEILA CHOKER'
selected_product_id=[Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]]
(selected_product_id[0])

'6878'

In [ ]:
# from Code.utils.libraries import * 
# from Code.data.dataprocessing.load.load import *
# from Code.data.dataprocessing.transform.transform import *
# from sessions import *
# from pyspark.sql.functions import date_format
# from Code.models.model_training.model import *


# BASE_PATH=os.getenv('BASEPATH')
# CONN_URL = os.getenv('CONN_URL')
# PASSWORD = os.getenv('PASSWORD')
# USERNAME=os.getenv('USER_NAME')
# configur = ConfigParser() 
# configur.read(BASE_PATH+'config.ini')
#spark_session=session(configur.get('driver_path_pyspark','pyspark'))
#configur.get('cosin_similarity_base_path','cs3'))#Sload_csv(spark=spark_session,file_path=configur.get('cosin_similarity_base_path','cs3'))

cosin_df=[]
product_title='BLUE CLUTCH'
product_ids=[Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]]#[(Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0])]#cs1.Product_A.unique()#[id[0] for id in cs1.select('Product_A').distinct().collect()]  in pyspark [8692,2178,8687,1087,3069]
print("total number of product",len(product_ids))
# st = time.time()

for r in product_ids:
    r=int(r)
    print(r)
    cs1_product_b=[]
    cs2_product_b=[]
    cs3_product_b=[]

    cs1_product_b=cs1[(cs1['Product_A']==r) & (cs1['cosin_score']>=.80)]['Product_B'].unique().tolist()  #[ product[0] for product in cs1.filter((cs1['Product_A']==r) & (cs1['cosin_score']>=.80))['Product_B'].unique()]
    cs2_product_b=cs2[(cs2['Product_A']==r) & (cs2['cosin_score']>=.85)]['Product_B'].unique().tolist()#values
    cs3_product_b=cs3[(cs3['Product_A']==r) & (cs3['cosin_score']>=.90)]['Product_B'].unique().tolist()#values
    # cs2temp=cs2.filter((cs2['Product_A']==r) & (cs2['cosin_score']>=.85))
    # cs3temp=cs3.filter((cs3['Product_A']==r) & (cs3['cosin_score']>=.90))
    # print(cs1_product_b)
    # print(cs2_product_b)
    # print(cs3_product_b)

    # for row in  cs1temp.collect():
    #     if float(row['cosin_score'])>=.80:
    #         cs1_product_b.append(row['Product_B'])
        
    # for row2 in cs2temp.collect():
    #     if float(row2['cosin_score'])>=.85:
    #         cs1_product_b.append(row2['Product_B'])
    #         # cs2_product_b.append(row2['Product_B'])
    # for row3 in cs3temp.collect():
    #     if float(row3['cosin_score'])>=.90 :
    #         cs1_product_b.append(row3['Product_B'])
                # cs3_product_b.append(row3['Product_B'])

#print(cs1_product_b,cs2_product_b,cs3_product_b)
    for id in cs2_product_b:
        cs1_product_b.append(id)
    for id in cs3_product_b:
        cs1_product_b.append(id)
    # final_list=cs1_product_b+cs2_product_b+cs3_product_b
    my_dict = {i:cs1_product_b.count(i) for i in cs1_product_b}
    # print(my_dict)
    top_recomendation=[]
    for key,value in my_dict.items():
        if value >=2:
            top_recomendation.append(key)
    print('top re',top_recomendation)
    cosin_df.append(cs1[(cs1['Product_A']==r) & (cs1['Product_B'].isin(top_recomendation))])
    # cosin_df.append(cs1.filter((cs1['Product_A']==r) & (cs1['Product_B'].isin(top_recomendation))))  # in pandas

df=cosin_df[0]
print(len(cosin_df[1:]))
for i in cosin_df[1:]:
    #  try:
         #df=df.union(i) #in pyspark
         df=df._append(i,ignore_index = True) #in pandas
    #  except:
    #      print('out of index')
#df.toPandas().to_csv('cosin_simmilarity1.csv')
# et = time.time()
# print(f'Execution time:{et-st} seconds')
# df.to_csv('cosin_simmilarity2.csv')






total number of product 1
6878
top re [3638, 3575, 3932, 1949, 5715, 5279, 5089, 4532, 3554, 4103, 2659, 557, 4292, 764, 2394, 915, 4142, 3637, 3725, 5281, 4718, 3211, 6095, 2656, 5171, 5126, 4145, 2399, 4734, 2983, 5466, 2969, 2620, 6265, 5363, 5131, 5127, 8867, 4106, 2590, 1324, 8862, 2398, 6880, 8479, 8133]
0
